#**Deep Natural Language Processing @ PoliTO**

---


**Teaching Assistant:** Moreno La Quatra

**Practice 1:** Text processing and topic modeling

# **Text processing**
---
The text processing phase is a preliminary stage where the text to be manipulated is processed to be ready for subsequent analysis.

Text processing usually entails several steps that could possibly include:
- **Language Identification**: identifying the language of a given text.
- **Tokenization**: splitting a given text in several sentences/words. 
- **Dependency tree parsing:** analyzing the depencies between words composing the text.
- **Stemming/Lemmatization:** obtain the root form for each word in text.
- **Stopword removal**: removing words that are si commonly used that they carry very little useful information.
- **Part of Speech Tagging:** given a word, retrieve its part of speech (proper noun, common noun or verb).



### Language Identification

| Text                                                                                                                                | Language Code |
|-------------------------------------------------------------------------------------------------------------------------------------|---------------|
| The "Deep Natural Language Processing" course is offered during the first semester of the second year at Politecnico di Torino      | `EN`            |
| Il corso "Deep Natural Language Processing" viene impartito al Politecnico di Torino durante il primo semestre del secondo anno.    | `IT`            |
| Le cours "Deep Natural Language Processing" est enseigné au Politecnico di Torino pendant le premier semestre de la deuxième année. | `FR`            |

**Language Identification** is a crucial prelimiary step because each language has its own characteristics. The knowledge of the main language associated to a given text could be beneficial for all subsequent steps in text processing pipeline.

The data collection used in this first part of the practice is provided [here](https://github.com/MorenoLaQuatra/DeepNLP/blob/main/practices/P1/langid_dataset.csv) - [source: Kaggle](https://www.kaggle.com/martinkk5575/language-detection)


# Exercise 1:

Benchmark different language-detection algorithm by computing the accuracy of each approach:
- [FastText](https://pypi.org/project/fastlangid/)
- [LangID](https://github.com/saffsd/langid.py)
- [langdetect](https://pypi.org/project/langdetect/)

**Hint:** language code conversion: [iso639-lang](https://pypi.org/project/iso639-lang/)

For each method report:
- Accuracy
- Average time per example

In [ ]:
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/langid_dataset.csv

In [ ]:
%%capture
! pip install fastlangid
! pip install langid
! pip install langdetect
! pip install iso639-lang
! pip install pandas --upgrade
! pip install numpy --upgrade

In [ ]:
# dataset reading 
import pandas as pd
from iso639 import Lang
df_langid = pd.read_csv('langid_dataset.csv')
print (df_langid)
# convert to language codes
df_langid['language'] = df_langid['language'].apply(lambda x: Lang(x).pt1)
print (df_langid)

In [ ]:
! pip install sklearn # to compute accuracy
! pip install tqdm # advancement

In [ ]:
import langid
from tqdm import tqdm
import time
from sklearn.metrics import accuracy_score
y_true = []
y_pred = []

start = time.time()
for index, row in tqdm(df_langid.iterrows()):
    sentence = row["Text"]
    real_lang_code = row["language"]
    y_true.append(real_lang_code)
    y_pred.append(langid.classify(sentence)[0])
end = time.time()

print ("\nlangid avg ms per example:", (end-start)*1000/len(df_langid.index))
print ("\nlangid accuracy:", accuracy_score(y_true, y_pred))

In [ ]:
from langdetect import detect

y_true = []
y_pred = []

start = time.time()
for index, row in tqdm(df_langid.iterrows()):
    sentence = row["Text"]
    real_lang_code = row["language"]
    y_true.append(real_lang_code)
    try:
        y_pred.append(detect(sentence))
    except Exception as e:
        y_pred.append("")
        print (e, "\n")
end = time.time()

print ("\nlangdetect avg ms per example:", (end-start)*1000/len(df_langid.index))
print ("\nlangdetect accuracy:", accuracy_score(y_true, y_pred))

In [ ]:
from fastlangid.langid import LID
ft_langid = LID()

y_true = []
y_pred = []

start = time.time()
for index, row in tqdm(df_langid.iterrows()):
    sentence = row["Text"]
    real_lang_code = row["language"]
    y_true.append(real_lang_code)
    try:
        y_pred.append(ft_langid.predict(sentence))
    except Exception as e:
        y_pred.append("")
        print (e, "\n")
end = time.time()

print ("\nlangdetect avg ms per example:", (end-start)*1000/len(df_langid.index))
print ("\nlangdetect accuracy:", accuracy_score(y_true, y_pred))

# Exercise 2

For English-written text, apply word-level tokenization. What is the average number of words per sentence?

Implement word-tokenization using both [nltk](https://www.nltk.org/) and [spacy](https://spacy.io/). Report the results for both of them.

For spaCy use the `en_core_web_sm` model.

In [ ]:
%%capture
! pip install --upgrade spacy
! pip install nltk
! python -m spacy download en_core_web_sm

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

len_total = 0
count_total = 0
for index, row in tqdm(df_langid.iterrows()):
    if row["language"] == "en":
        len_total += len(word_tokenize(row["Text"]))
        count_total += 1
print ("\nAvg. len in #words - NLTK:", len_total/count_total)

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

len_total = 0
count_total = 0
for index, row in tqdm(df_langid.iterrows()):
    if row["language"] == "en":
        doc = nlp(row["Text"])
        len_total += len(doc)
        count_total += 1
print ("\nAvg. len in #words - spaCy:", len_total/count_total)

# Exercise 3

Use spacy to parse the dependency tree of a **randomly selected** sentence. You can both use English sentences or your native language (if supported in [spaCy](https://spacy.io/usage/models/)). Use [displaCy](https://explosion.ai/demos/displacy) to visualize the result in the notebook.

In [ ]:
%%capture
! pip install --upgrade spacy
! python -m spacy download ro_core_news_sm
! python -m spacy download en_core_web_sm
! python -m spacy download pt_core_news_sm
! python -m spacy download it_core_news_sm

In [ ]:
import random
import spacy
sentences_pool = []
lang_code = "en"
if lang_code =="en":
    nlp = spacy.load(lang_code + '_core_web_sm')
else:
    nlp = spacy.load(lang_code + '_core_news_sm')

for index, row in df_langid.iterrows():
    if row["language"] == lang_code:
        sentences_pool.append(row["Text"])

s = random.choice(sentences_pool)
doc = nlp(s)
spacy.displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

# Exercise 4
For the same sentence selected in the previous step apply all the following steps:
1. Lemmatization: convert each word to its root form.
2. Stopword removal: remove language-specific stopwords.
3. Part of Speech Tagging: for each word in the sentence display its part-of-speech.

For each step, print the resulting list on the console.

In [ ]:
#Lemmatization
doc = nlp(s)
lemmas = []
for w in doc:
    lemmas.append(w.lemma_)
print(lemmas)

In [ ]:
#Stopword removal
clean_sentence = [] 
for w in doc:
    if not w.is_stop:
        clean_sentence.append(w.text)
print(" ".join(clean_sentence))

In [ ]:
#PoS Tagging
for w in doc:
    print (w.text, w.pos_)

# **Occurrence-based text representation - TF-IDF**

---

TF-IDF (term frequency-inverse document frequency) is a statistical measure that evaluates how relevant a word is to a document in a collection of documents. It allows to create occurrence-based vector representation for each document.

# Exercise 5
Use TF-IDF to vectorize each sentence in the original data collection. You can choose your preferred implementation for TF-IDF vectorization. It is also available on [SciKit-Learn library](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df_langid["Text"].tolist()
language_labels = df_langid["language"].tolist()

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

# Exercise 6

Build a supervised multi-class language detector using as features the vector obtained by TF-IDF representation. Use 80% of the data to train the language detector and 20% of the data for assessing its accuracy.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, language_labels , test_size=0.20)
mlp = MLPClassifier(verbose=True)
mlp = mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, language_labels , test_size=0.20)
dt = DecisionTreeClassifier()
dt = dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
accuracy_score(y_test, y_pred)

# **Topic Modelling**
---
Occurrence-based representations are high-dimensional, what is the dimension of the generated TF-IDF vector representation?
Topic modelling focuses on caturing latent topics in large document corpora.

The data collection used in this second part of the practice is provided [here](https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/CovidFake_filtered.csv) - [source: Zenodo](https://zenodo.org/record/4282522#.YVdCXcbOOpd)


# Exercise 7

Latent Semantic Indexing (LSI) models underlying concepts by using SVD (Singular Value Decomposition).

Use [gensim](https://radimrehurek.com/gensim/) library to:
1. Create a corpus composed of the headlines contained in the data collection.
2. Generate a [dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html) to create a word -> id mapping (required by LSI module).
3. Using the dictionary, preprocess the corpus to obtain the representation required for LSI model training ([documentation here](https://radimrehurek.com/gensim/models/lsimodel.html)).
4. Inspect the top-5 topics generated by the LSI model for the analysed corpus.

In [ ]:
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/CovidFake_filtered.csv

In [ ]:
import pandas as pd
df_tmodelling = pd.read_csv('CovidFake_filtered.csv')

In [ ]:
# generating the corpus 
corpus = df_tmodelling["headlines"].tolist()
corpus = [s.split() for s in corpus]
print (corpus[:10])

In [ ]:
# constructing dictionary and processing the corpus
from gensim.corpora.dictionary import Dictionary
tm_dict = Dictionary(corpus)
processed_corpus = [tm_dict.doc2bow(text) for text in corpus]

In [ ]:
# training LSI model
from gensim.models import LsiModel
model = LsiModel(processed_corpus, id2word=tm_dict)
model.print_topics(5)

# Exercise 8 (Optional)

The top-scored words contributing to each topic (if no stopword removal is applied) are english common words (e.g., *to, for, in, of, on*..). Moreover, missing punctuation removal could be critical for topic identification. Repeat the same procedure of Ex. 7 by adding preliminary preprocessing step to:
1. **remove stopwords**
2. **strip punctuation**
3. **lowercase all words**

In [ ]:
from nltk.corpus import stopwords
import nltk
import string

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
rs_corpus = [[w.lower() for w in s if w.lower() not in stop_words] for s in corpus]
rs_corpus = [[w.translate(str.maketrans('', '', string.punctuation)) for w in s] for s in rs_corpus]
print(rs_corpus[:10])

In [ ]:
rs_tm_dict = Dictionary(corpus)
rs_processed_corpus = [rs_tm_dict.doc2bow(text) for text in rs_corpus]
rs_model = LsiModel(rs_processed_corpus, id2word=rs_tm_dict)
rs_model.print_topics(5)

# Exercise 9 (Optional)

Leveraging the same corpus used for LSI model generation, apply LDA modelling setting the number of topics to 5. Display the words most contributing to the those topics according to the LDA model.

In [ ]:
from gensim.models.ldamodel import LdaModel
lda = LdaModel(rs_processed_corpus, id2word=rs_tm_dict, num_topics=3)
lda.print_topics(5)

# Exercise 10 (Optional)

Using [pyLDAvis]() library build an interactive visualization for the trained LDA model.

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

lda_display = gensimvis.prepare(lda, rs_processed_corpus, rs_tm_dict, sort_topics=False)
pyLDAvis.display(lda_display)